# zad.9 (lekko zmodyfikowane)

Zapoznaj się z wynikami z artykułu:\
*Mendonça, J. R. G. (2019).* **Simply modified GKL density classifiers that reach consensus faster.** *Physics Letters A, 383(19), 2264–2266. 
doi:10.1016/j.physleta.2019.04.033* (https://sci-hub.se/https://www.sciencedirect.com/science/article/abs/pii/S0375960119303512?via=ihub)

Spróbuj powtórzyć eksperymenty tam wykonane dla wybranego N dla którego mamy "zbiór Marcina" % poprawnych klasyfikacji dla reguł GKL(j, k) dla kilku, wybranych przez Ciebie przypadków j, k. **Wariant (*) - zbadaj również średni czas klasyfikacji - tzn. czy widać, które reguły są szybsze od innych.**

---
# zad.11

Zaimplementuj algorytm ewolucyjny, który poszukuje rozwiązań DCP wśród binarnych automatów komórkowych o promieniu sąsiedztwa r=2, dla liczby komórek N=23 (korzystając ze zbioru Marcina) i limitu czasu T = 2*N. Zdecyduj sama / sam ile populacji i jak duże populacje potrzebujesz - możesz to określić robiąc eksperymenty. Możesz również dowolnie manipulować mutacjami, krzyżowaniem, selekcją itd.

Jaki najlepszy wynik (największą liczbę poprawnie sklasyfikowanych konfiguracji) uda Ci się uzyskać? Podaj LUT najlepszej, znalezionej przez Twój algorytm reguły. 

Uzyskane przez nas wszystkich reguły porównam na zbiorach N=23 i większych, jak tylko otrzymam wszystkie rozwiązania i wyłonimy zwycięzców. Może uda się komuś z nas pobić GKL?

Sugestie:
- Do wstępnych testów możesz najpierw użyć mniejszego N i zamiast zbioru Marcina, użyć mniejszy zbiór wygenerowany losowo - program będzie działać szybciej, zatem szybciej będziesz wprowadzać w nim zmiany.
- Koniecznie użyj przetwarzania równoległego aby w pełni wykorzystać moc współczesnych procesorów oraz cache’owania - tak, żeby nie przeliczać tej samej reguły CA wiele razy.
- GitHub CodeSpaces dają 4-rdzeniowy procesor w chmurze dostępny za darmo (30 lub 60 godzin obliczeń w miesiącu).
- Czy dla każdego wyliczenia wartości funkcji dopasowania trzeba używać cały zbiór warunków początkowych? Może aby było szybciej, dla każdej populacji można losować pewien podzbiór zbioru warunków początkowych i dzięki temu szybciej tworzyć nowe populacje? (A może te podzbiory też powinny podlegać “ewolucji”? Mamy wtedy do czynienia z koewolucją.)
- Przedyskutuj to zagadnienie z ChatGPT. Poproś o sugestie, pomysły na usprawnienia kodu itd!